# Metabolism Circle Heatmap

This notebooks takes the genes that are a hit from the Brca NCI-Nature_2016 (PDGFR-beta pathway) and maps them on a large circle heat map. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load df with all of the genes that are FDR significant. Then get list of just the gene names and use them to run a GSEA. 

In [2]:
prot_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Brca_EGFR_all_pearson_FDR.csv")
df_FDR= prot_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,MPP6_proteomics,CPNE8_proteomics,CRYBG3_proteomics,PSAT1_proteomics,PHC3_proteomics,GATA3_proteomics,KRT16_proteomics,KRT5_proteomics,NXN_proteomics,...,TSC1_proteomics,MCM5_proteomics,HNRNPA3_proteomics,GC_proteomics,LASP1_proteomics,GGH_proteomics,KIF15_proteomics,MNS1_proteomics,RRP1_proteomics,ZDHHC20_proteomics
Correlation,1.0,6.482017e-01,6.164527e-01,6.099971e-01,6.093187e-01,-6.032044e-01,-5.989113e-01,5.953255e-01,5.947704e-01,5.928643e-01,...,-0.229952,0.229894,-0.229883,0.229882,-0.229848,0.229847,0.229837,0.270088,0.229647,0.229381
P_value,0.0,4.834438e-15,2.229263e-13,4.612176e-13,4.973595e-13,9.738761e-13,1.547846e-12,2.267369e-12,2.404384e-12,2.938573e-12,...,0.013427,0.013451,0.013455,0.013456,0.013470,0.013470,0.013475,0.013535,0.013554,0.013665


In [3]:
brca_prot = df1_transposed.columns.values.tolist()
brca_genes = []
for gene in brca_prot :
    brca_genes.append((re.sub("_proteomics", "", gene)))
len(brca_genes)

2669

Run GSEA using reactome 2016 set

In [4]:
brca_enr = gp.enrichr(gene_list = brca_genes, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='test/enrichr_kegg')
brca_enr.res2d.head(2)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,NCI-Nature_2016,Urokinase-type plasminogen activator (uPA) and...,25/42,3.110276e-12,6.500478e-10,0,0,4.460383,118.183685,ITGB1;ITGAM;LRP1;SRC;ITGB3;ITGB2;SERPINE1;PLG;...
1,NCI-Nature_2016,PDGFR-beta signaling pathway Homo sapiens c901...,47/128,2.028208e-11,2.119477e-09,0,0,2.751499,67.745429,USP6NL;DOCK4;TAGLN;LRP1;SRC;ARPC1B;ITGB3;BRK1;...


In [7]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_all_prot_append_return_all.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)


In [8]:
#get just the upa genes
brca_df = brca_enr.res2d
upa = brca_df.iloc[0,9]
upa = upa.split(';')
upa.remove("EGFR")
len(upa)
upa


['ITGB1',
 'ITGAM',
 'LRP1',
 'SRC',
 'ITGB3',
 'ITGB2',
 'SERPINE1',
 'PLG',
 'VLDLR',
 'VTN',
 'PLAU',
 'ITGAV',
 'ELANE',
 'FGB',
 'FGA',
 'TGFB1',
 'FGG',
 'MMP3',
 'FN1',
 'PLAUR',
 'MMP9',
 'MMP12',
 'ITGA5',
 'GPLD1']

In [9]:
#filter down df with just upa genes 
upa_column_names = []
for gene in upa:
    gene += "_proteomics"
    upa_column_names.append(gene)

df_FDR_upa = df_FDR_append[df_FDR_append.Comparison.isin(upa_column_names)]
df_FDR_upa

,Comparison,Correlation,P_value,Cancer Type
196,PLAUR_proteomics,-0.425639,0.000011,GBM
271,ITGB1_proteomics,-0.412364,0.000022,GBM
305,FGB_proteomics,-0.404936,0.000032,GBM
350,FGG_proteomics,-0.396275,0.000049,GBM
445,FGA_proteomics,-0.380889,0.000101,GBM
...,...,...,...,...
18448,ITGAV_proteomics,0.317536,0.000724,Lscc
18621,ITGB1_proteomics,0.288139,0.002269,Lscc
19224,ITGA5_proteomics,0.233016,0.014294,Lscc
19825,MMP3_proteomics,0.198863,0.037277,Lscc


In [10]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_upa, "P_value","Correlation","Comparison","Cancer Type",plot_width= 1000, plot_height = 650)

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [15]:
luad_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Luad_EGFR_all_pearson_FDR.csv")

In [17]:
df_FDR= luad_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,GGCT_proteomics,LANCL2_proteomics,NUDCD3_proteomics,TAX1BP1_proteomics,ADGRF1_proteomics,GRAP_proteomics,CDK13_proteomics,GRAMD2B_proteomics,MACC1_proteomics,...,WDR19_proteomics,SLC25A22_proteomics,OXSR1_proteomics,PCSK6_proteomics,TMEM205_proteomics,VGLL4_proteomics,ST5_proteomics,CSTF1_proteomics,SPRYD4_proteomics,LDB3_proteomics
Correlation,1.0,6.090062e-01,5.888081e-01,5.683828e-01,5.615545e-01,6.068124e-01,-5.441949e-01,5.413530e-01,5.621980e-01,5.222575e-01,...,-0.261754,0.261729,-0.261669,-0.324513,0.267685,0.261583,0.261481,0.261472,-0.261441,-0.278121
P_value,0.0,1.676601e-12,1.328507e-11,9.362813e-11,1.746498e-10,7.466411e-10,8.008013e-10,1.019153e-09,1.703448e-09,4.866322e-09,...,0.005740,0.005745,0.005756,0.005762,0.005768,0.005773,0.005793,0.005794,0.005801,0.005810


In [18]:
luad_prot = df1_transposed.columns.values.tolist()
luad_genes = []
for gene in luad_prot :
    luad_genes.append((re.sub("_proteomics", "", gene)))
len(luad_genes)

1198

In [25]:
luad_enr = gp.enrichr(gene_list = luad_genes, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='test/enrichr_kegg')
luad_enr.res2d.head(7)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Immune System Homo sapiens R-HSA-168256,158/1547,1.186818e-11,1.815831e-08,0,0,1.705061,42.894496,ATF1;CYFIP2;APP;AHCYL1;NCF1;WIPF1;PROS1;WIPF2;...
1,Reactome_2016,Innate Immune System Homo sapiens R-HSA-168249,93/807,8.043351e-10,6.153164e-07,0,0,1.923900,40.288409,ATF1;CYFIP2;APP;AHCYL1;CDKN1B;WIPF1;WIPF2;PROS...
2,Reactome_2016,RHO GTPases Activate WASPs and WAVEs Homo sapi...,14/36,7.762482e-09,3.958866e-06,0,0,6.492302,121.237012,CYFIP2;WIPF1;WIPF2;WAS;BRK1;ARPC4;ARPC5;PTK2;A...
3,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,66/552,6.232478e-08,2.383923e-05,0,0,1.996080,33.116784,APP;DOCK5;DGKB;PROS1;F13A1;PIK3CD;ARRB2;CLU;AK...
4,Reactome_2016,Membrane Trafficking Homo sapiens R-HSA-199991,54/420,9.540055e-08,2.919257e-05,0,0,2.146435,34.697504,APP;GCC1;USE1;GOLGA5;KIF13B;KIFAP3;VPS36;CTSC;...
5,Reactome_2016,"Platelet activation, signaling and aggregation...",38/253,1.539350e-07,3.925342e-05,0,0,2.507473,39.334065,CFD;APP;ITIH3;DGKB;PROS1;F13A1;PIK3CD;ARRB2;RA...
6,Reactome_2016,Signaling by Rho GTPases Homo sapiens R-HSA-19...,48/367,2.939688e-07,6.425318e-05,0,0,2.183476,32.839020,CYFIP2;TRIO;CDKN1B;NCF1;WIPF1;PKN3;WIPF2;SRF;F...


In [39]:
#get just the upa genes
luad_df = luad_enr.res2d
genes = luad_df.iloc[3,9]
genes = genes.split(';')

genes.remove("GRB2")

len(genes)
genes

['APP',
 'DOCK5',
 'DGKB',
 'PROS1',
 'F13A1',
 'PIK3CD',
 'ARRB2',
 'CLU',
 'AKAP10',
 'PPP2R5E',
 'LAMP2',
 'KIF13B',
 'RAC2',
 'KIFAP3',
 'JAK3',
 'HRAS',
 'SRGN',
 'GUCY1A2',
 'VWF',
 'PRKCB',
 'APLP2',
 'HGF',
 'SERPINF2',
 'ATP1B1',
 'APBB1IP',
 'FGR',
 'F9',
 'STIM1',
 'MMRN1',
 'LCK',
 'CEACAM6',
 'IRF7',
 'PRKCQ',
 'CD48',
 'PDE5A',
 'DAGLB',
 'TLN1',
 'PFN1',
 'ALDOA',
 'DOCK2',
 'CFD',
 'ITIH3',
 'AK3',
 'PRCP',
 'RASGRP2',
 'GNA14',
 'KLC3',
 'GNG2',
 'INPP5D',
 'PLCG2',
 'SPP2',
 'CSK',
 'CD74',
 'SERPIND1',
 'ANGPT1',
 'F12',
 'PTK2',
 'SELP',
 'CD2',
 'PROC',
 'P2RX1',
 'CD9',
 'F13B',
 'PTPN6',
 'HRG']

In [41]:
#filter down df with just upa genes 
pathway_names = []
for gene in genes:
    gene += "_proteomics"
    pathway_names.append(gene)

df_FDR_pathway = df_FDR_append[df_FDR_append.Comparison.isin(pathway_names)]
df_FDR_pathway

,Comparison,Correlation,P_value,Cancer Type
53,APBB1IP_proteomics,-0.483056,4.103970e-07,GBM
58,CSK_proteomics,-0.481565,4.507321e-07,GBM
73,PLCG2_proteomics,-0.472453,7.918099e-07,GBM
87,PIK3CD_proteomics,-0.466265,1.150323e-06,GBM
183,DOCK2_proteomics,-0.428841,9.481834e-06,GBM
...,...,...,...,...
8413,KLC3_proteomics,0.431755,2.483261e-06,Lscc
8495,KIF13B_proteomics,-0.377294,4.842222e-05,Lscc
8547,CD9_proteomics,0.354370,1.457314e-04,Lscc
8632,STIM1_proteomics,0.326801,4.929629e-04,Lscc


In [42]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_pathway, "P_value","Correlation","Comparison","Cancer Type",plot_width= 1000, plot_height = 650)

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...